# Lab 2

Implement an extension to the classifier from Part 1.2, in which you also use features based on continuous vector representations of words/phrases in addition to the already implemented linguistic/lexical feature

* Compare performance between extended classifier and basic classifier of Lab 1 
* Compare performance between extended classifier and classifier based **only** on continuous representation 

Three different classifiers:
* Lab 1.2: Based on sparse vectors
* Based on continuous representations using a word embedding 
* Combination of the two, maybe combining tfidf with word embedding (Why?: Token embeddings treat every token equally, though they might not be of the same importance for the meaning in the document --> Since we work with embedding documents (by pooling by mean), we want to weigh the different words by their tfidf score when calculating the mean of a doc)

Take-away Lab1:
* English:
  - Nltk, question_context, no preprocessing, tfidf weighting (f1 = 0.759152, acc = 0.747475)
* Finnish:
  - Nltk, context, no prep, no weighting (f1 = 0.738518, acc = 0.746738)
  - Nltk, question_context, no prep, no or tfidf weighting (bit worse performance but still quite good compared to the other models)
* Japanese:
  - Bert, context, no prep, tfidf weighting (f1 = 0.713147, acc = 0.722008)
  - Fugashi, question_context, prep, tfidf weighting (f1 = 0.717162, acc = 0.715251)

--> Here, we are just going to use the concatenated question + context as input based on the results of Lab 1 as well as the outlook on the following lab sessions.

In [ ]:
!pip install nltk
!pip install spacy
!pip install transformers
!pip install pyyaml
!pip install datasets
!pip install bpemb
!pip install gensim
!pip install fugashi
!pip install googletrans
!python -m spacy download fi_core_news_sm
!python -m spacy download ja_core_news_sm
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 14.7 MB/s 
     |████████████████████████████████| 7.6 MB 41.8 MB/s 
     |████████████████████████████████| 163 kB 53.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 14.5 MB/s 
     |████████████████████████████████| 212 kB 68.7 MB/s 
     |████████████████████████████████| 115 kB 53.5 MB/s 
     |████████████████████████████████| 95 kB 6.3 MB/s 
     |████████████████████████████████| 127 kB 66.7 MB/s 
     |████████████████████████████████| 115

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
!unzip wiki-news-300d-1M.vec.zip

!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.fi.vec
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.ja.vec 

--2022-10-24 13:58:16--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  3.32MB/s    in 4m 21s  

2022-10-24 14:02:39 (2.49 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]

Archive:  wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   
--2022-10-24 14:02:57--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.fi.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awai

In [ ]:
import spacy
import nltk
from bpemb import BPEmb
from transformers import AutoTokenizer
import numpy as np
from datasets import load_dataset
import spacy
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import fugashi
import matplotlib.pyplot as plt
import time
from googletrans import Translator
from bpemb import BPEmb
import gensim
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Functions

In [ ]:
def loadDataset():
  """ Returns the raw TyDiQa dataset, only including the languages English, Finnish, and Japanese
  Two columns are added: 
      A binary column to indicate whether an answer is available or not
      The concatenated question and context
  Returns the data split into training and validation data"""
  dataset = load_dataset("copenlu/answerable_tydiqa")
  dataset = dataset.filter(lambda x: x["language"] == "english" or x["language"] == "finnish" or x["language"] == "japanese")
  train_set = dataset["train"]
  validation_set = dataset["validation"]
  training_data = answer_available(pd.DataFrame.from_dict(train_set))
  validation_data = answer_available(pd.DataFrame.from_dict(validation_set))
  training_data['question_context'] = training_data['question_text'] + ' ' + training_data['document_plaintext']
  validation_data['question_context'] = validation_data['question_text'] + ' ' + validation_data['document_plaintext']
  return training_data, validation_data


def answer_available(df):
    """Add a column indicating whether an answer is available in the context or not
    0: no answer available, 1: answer available"""
    df['answer_available'] = [0 if df['annotations'][i]['answer_text'] == [''] else 1 for i in range(len(df))]
    return df

def split_data(train_df, val_df, language):
    """Split the training and validation data into the three languages"""
    train = train_df[train_df["language"] == language].reset_index(drop = True)
    val = val_df[val_df["language"] == language].reset_index(drop = True)
    return train, val

def preprocessing(text, low = False, footnotes = True, punct = True, whitespaces = True):
    """Preprocess text
    Following arguments set to True by default except for lowercase: 
        Convert everything to lowercase 
        Remove Wikipedia footnotes
        Remove punctuation 
        Remove encoded and / or multiple whitespaces
    Always: Remove special quotationsmarks (found in japanese text)"""
    if low == True:
        text = text.apply(lambda x: x.lower())
    if footnotes == True:
        text = re.sub('\[[0-9]+\]', '', text)
    if punct == True:
        text = text.translate(str.maketrans('', '', string.punctuation))
    if whitespaces == True:
        text = re.sub('\s+', ' ', text)
    text = re.sub('”', '', text)
    return text

def do_nothing(tokens):
    """Placeholder function for the CountVectorizer and TfidfVectorizer by sklearn"""
    return tokens

def tokenizeData(data, tokenizer, language, preprocessor = False):
    """Takes any iterable as input return lists of tokens in a list
    data: any interable
    tokenizer: bert, spacy, nltk, fugashi
    language: english, finnish, japanese
    preprocessor: if True execute preprocessing as in the function above"""
    data = data['question_context']
    if preprocessor == True:
        data = data.apply(lambda x: preprocessing(x))
    if tokenizer == 'spacy':
        if language == 'english':
            tokenizer = spacy.load('en_core_web_sm')
        elif language == 'finnish':
            tokenizer = spacy.load("fi_core_news_sm")
        elif language == 'japanese':
            tokenizer = spacy.load('ja_core_news_sm')
        output_data = data.apply(lambda x: [token.text for token in tokenizer(x)])
    elif tokenizer == 'nltk':
        if language == 'english':
            output_data = data.apply(lambda x: word_tokenize(x, language = 'english'))
        elif language == 'finnish':
            output_data = data.apply(lambda x: word_tokenize(x, language = 'finnish'))
        elif language == 'japanese':
            print('NLTK does not support tokenization in japanese.')
            output_data = None
    elif tokenizer == 'fugashi':
        if language == 'japanese':
            tagger = fugashi.Tagger()
            output_data = data.apply(lambda x: [token.surface for token in tagger(x)])
        else:
            print('Fugashi only supports the language Japanese.')
            output_data = None
    return output_data

def vectorize_tokens(training_tokens, validation_tokens, tfidf = False):
    """Vectorize tokens using either count values or tfidf weights if tfidf is set to True"""
    # Count vs. Tfidf vectorizer
    if tfidf == False:
        vectorizer = CountVectorizer(tokenizer = do_nothing, 
                                    preprocessor = do_nothing)
    else:
        vectorizer = TfidfVectorizer(tokenizer = do_nothing, 
                                    preprocessor = do_nothing)
    
    # Creating dfm
    vectorizer = vectorizer.fit(training_tokens)
    X_train = vectorizer.transform(training_tokens)
    X_val = vectorizer.transform(validation_tokens)
    return X_train, X_val

def embed_tokens(train_toks, val_toks, language, tfidf = True):
    """Vectorize tokens using FastText word embedding
    train_toks: training tokens
    val_toks: validation tokens
    language: english, finnish, japanese
    tfidf: When pooling the embedding matrix, compute the average using the tfidf scores as weights
          If tfidf is set to False all words in a document count equally when computing the mean
    Returns pooled mean vector per document"""

    # Loading the model
    if language == 'english':
        fasttext = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec', binary=False)
    elif language == 'finnish':
        fasttext = KeyedVectors.load_word2vec_format('wiki.fi.vec', binary=False)
    elif language == 'japanese':
        fasttext = KeyedVectors.load_word2vec_format('wiki.ja.vec', binary=False)

    # Applying vectorizer
    if tfidf == True:
        vectorizer = TfidfVectorizer(tokenizer = do_nothing, 
                                preprocessor = do_nothing)
    else:
        vectorizer = CountVectorizer(tokenizer = do_nothing, 
                                     preprocessor = do_nothing)

    # Creating matrices with either count values or tfidf weights of features    
    vectorizer = vectorizer.fit(train_toks)
    X_train = vectorizer.transform(train_toks)
    X_val = vectorizer.transform(val_toks)
    
    # Create embedding matrix
    features = vectorizer.get_feature_names_out() # Get vocab
    unk = fasttext.vectors.mean(axis = 0) # Create vector for oov tokens: mean of all embedding vectors to account for distribution
    embedding_matrix=np.zeros((len(features), 300)) # create empty matrix
    
    # Fill the matrix with respective embedding of token
    for i in range(len(features)):
        try:
            embedding_matrix[i]=fasttext.get_vector(features[i])
        # If token not known by embedding, fill with OOV vector
        except KeyError:
            embedding_matrix[i] = unk

    # Create matrix of pooled doc embedding of size len of docs x dim of embedding
    doc_train = np.zeros((len(train_toks), 300))

    for i in range(len(train_toks)):
        # Fill every row with averaged embedding
        doc_train[i]= np.average(embedding_matrix, axis = 0, weights = X_train[i].toarray()[0])

    doc_val = np.zeros((len(val_toks), 300))

    for i in range(len(val_toks)):
        doc_val[i] = np.average(embedding_matrix, axis = 0, weights = X_val[i].toarray()[0])

    return doc_train, doc_val 


def lr_classifier(X_train, X_val, y_train, y_val):
    
    # Training LogisticRegression, random_state set to 42 
    log_mod = LogisticRegression(random_state = 42, max_iter = 1000).fit(X_train, y_train)
    
    # Evaluating model
    preds = log_mod.predict(X_val)
    f1 = f1_score(y_val, preds)
    acc = accuracy_score(y_val, preds)
    print(f'F1 = {f1}')
    print(f'Accuracy = {acc}')
    print(classification_report(y_val, preds))
    return f1, acc

def test_classifier(train_data, val_data, language, sparse = True, continuous = True, tokenizer = None):
    """Putting it all together
    sparse and continuous both set to True will combine the linguistic assumptions by tfidf weighting
        with continuous representations of words by word embeddings
    Only continuous set to True will use only continuous word representations using word embeddings
    Only sparse set to True will use only sparse word vectors using tfidf weighting"""

    y_train = np.array(train_data['answer_available'])
    y_val = np.array(val_data['answer_available'])

    if continuous == True:

        if language == 'english':
                train_tok = tokenizeData(train_data, tokenizer = 'nltk', language = language, preprocessor = False)
                val_tok = tokenizeData(val_data, tokenizer = 'nltk', language = language, preprocessor = False)
                
        elif language == 'finnish':
                train_tok = tokenizeData(train_data, tokenizer = 'nltk', language = language, preprocessor = False)
                val_tok = tokenizeData(val_data, tokenizer = 'nltk', language = language, preprocessor = False)

        elif language == 'japanese':
                train_tok = tokenizeData(train_data, tokenizer = 'fugashi', language = language, preprocessor = True)
                val_tok = tokenizeData(val_data, tokenizer = 'fugashi', language = language, preprocessor = True)

        if sparse == True:
            print('Binary classifier combining linguistical features and continuous word representations')
            X_train, X_val = embed_tokens(train_tok, val_tok, language, tfidf = True)
            f1, acc = lr_classifier(X_train, X_val, y_train, y_val)

        else:
            print('Binary classifier using continuous word representations')
            X_train, X_val = embed_tokens(train_tok, val_tok, language, tfidf = False)
            f1, acc = lr_classifier(X_train, X_val, y_train, y_val)
    
    elif sparse == True:
        print('Binary classifier using only linguistical features')
        if language == 'english':
            print(f"Tokenizer: Nltk, no preprocessing, tfidf weighting")
            train_tok = tokenizeData(train_data, tokenizer = 'nltk', language = language, preprocessor = False)
            val_tok = tokenizeData(val_data, tokenizer = 'nltk', language = language, preprocessor = False)
            X_train, X_val = vectorize_tokens(train_tok, val_tok, tfidf = True)
            f1, acc = lr_classifier(X_train, X_val, y_train, y_val)
        elif language == 'finnish':
            print(f"Tokenizer: Nltk, no preprocessing, tfidf weighting")
            train_tok = tokenizeData(train_data, tokenizer = 'nltk', language = language, preprocessor = False)
            val_tok = tokenizeData(val_data, tokenizer = 'nltk', language = language, preprocessor = False)
            X_train, X_val = vectorize_tokens(train_tok, val_tok, tfidf = True)
            f1, acc = lr_classifier(X_train, X_val, y_train, y_val)
        elif language == 'japanese':
            print(f"Tokenizer: Fugashi, preprocessing, tfidf weighting")
            train_tok = tokenizeData(train_data, tokenizer = 'fugashi', language = language, preprocessor = True)
            val_tok = tokenizeData(val_data, tokenizer = 'fugashi', language = language, preprocessor = True)
            X_train, X_val = vectorize_tokens(train_tok, val_tok, tfidf = True)
            f1, acc = lr_classifier(X_train, X_val, y_train, y_val)
    
    return f1, acc
    

In [ ]:
# Load data

training_data, validation_data = loadDataset()
# Split by language
train_en, val_en = split_data(training_data, validation_data, 'english')
train_fi, val_fi = split_data(training_data, validation_data, 'finnish')
train_ja, val_ja = split_data(training_data, validation_data, 'japanese')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/117 [00:00<?, ?ba/s]

In [ ]:
results = pd.DataFrame(columns = ['language', 'model', 'input', 'tokenizer', 'f1', 'acc'])

for lan in ['english', 'finnish', 'japanese']:
    if lan == 'english':
        train = train_en
        val = val_en
    elif lan == 'finnish': # weird result for sparse and finnish
        train = train_fi
        val = val_fi
    elif lan == 'japanese':
        train = train_ja
        val = val_ja

    f1, acc = test_classifier(train, val, language = lan)
    results.loc[len(results)] = [f'{lan}', 'combined', 'question_context', 'nltk', f1, acc]

    f1, acc = test_classifier(train, val, language = lan, sparse = False)
    results.loc[len(results)] = [f'{lan}', 'continuous', 'question_context', 'nltk', f1, acc]

    # Do this on jupyter notebook with fugashi dict
    if lan != 'japanese': 
        f1, acc = test_classifier(train, val, language = lan, continuous = False)
        results.loc[len(results)] = [f'{lan}', 'sparse', 'question_context', 'nltk', f1, acc]

Binary classifier combining linguistical features and continuous word representations


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


F1 = 0.6959775491113189
Accuracy = 0.6717171717171717
              precision    recall  f1-score   support

           0       0.70      0.59      0.64       495
           1       0.65      0.75      0.70       495

    accuracy                           0.67       990
   macro avg       0.68      0.67      0.67       990
weighted avg       0.68      0.67      0.67       990

Binary classifier using continuous word representations


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


F1 = 0.711645101663586
Accuracy = 0.6848484848484848
              precision    recall  f1-score   support

           0       0.73      0.59      0.65       495
           1       0.66      0.78      0.71       495

    accuracy                           0.68       990
   macro avg       0.69      0.68      0.68       990
weighted avg       0.69      0.68      0.68       990

Binary classifier using only linguistical features
Tokenizer: Nltk, no preprocessing, tfidf weighting


...

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


F1 = 0.7100656455142232
Accuracy = 0.6856465005931198
              precision    recall  f1-score   support

           0       0.72      0.60      0.66       843
           1       0.66      0.77      0.71       843

    accuracy                           0.69      1686
   macro avg       0.69      0.69      0.68      1686
weighted avg       0.69      0.69      0.68      1686

Binary classifier using only linguistical features
Tokenizer: Nltk, no preprocessing, tfidf weighting


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


F1 = 0.7329902801600915
Accuracy = 0.7230130486358244
              precision    recall  f1-score   support

           0       0.74      0.69      0.71       843
           1       0.71      0.76      0.73       843

    accuracy                           0.72      1686
   macro avg       0.72      0.72      0.72      1686
weighted avg       0.72      0.72      0.72      1686



RuntimeError: ignored

In [ ]:
from google.colab import files
results.to_csv('resultsfasttext_lab2.csv')
files.download('resultsfasttext_lab2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
results_ja = pd.DataFrame(columns = ['language', 'model', 'input', 'tokenizer', 'f1', 'acc'])

lan = 'japanese'

#f1, acc = test_classifier(train_ja, val_ja, language = lan)
results_ja.loc[len(results_ja)] = [f'{lan}', 'combined', 'question_context', 'fugashi', f1, acc]

f1, acc = test_classifier(train_ja, val_ja, language = lan, sparse = False)
results_ja.loc[len(results_ja)] = [f'{lan}', 'continuous', 'question_context', 'fugashi', f1, acc]

f1, acc = test_classifier(train_ja, val_ja, language = lan, continuous = False)
results_ja.loc[len(results_ja)] = [f'{lan}', 'sparse', 'question_context', 'fugashi', f1, acc]

In [ ]:
results_first = pd.read_csv('resultsfasttext_lab2.csv')
results = pd.concat([results_first, results_ja])
results.to_csv('final_results_lab2.csv')